*House Pricing*

*Fase 1 - Pré-processamento e transformação*

**Grupo:** 
* Carlos Gabriel de Freitas - 19.1.4009
* Gabriel Mace dos Santos Ferreira - 19.1.4013
* Filipe Ramos de Souza Santo - 19.1.4027

**Disciplina:** Mineração de Dados (BCC444)

**Professor:** Anderson Almeida Ferreira

In [ ]:
attributes = [
    {
        'name': 'Id',
        'category': 'discrete_numerical',
        'description': 'Identificação de cada residência no data sheet',
    },
    {
        'name': 'MSSubClass',
        'category': 'nominal_categorical',
        'description': 'Identifica a classificação geral do zoneamento da venda',
    },
    {
        'name': 'MSZoning',
        'category': 'nominal_categorical',
        'description': 'Identifica a classificação geral do zoneamento da venda',
    },
    {
        'name': 'LotFrontage',
        'category': 'continuous_numerical',
        'description': 'Comprimento total das ruas conectadas à propriedade (medido em Linear Feet)',
    },
    {
        'name': 'LotArea',
        'category': 'continuous_numerical',
        'description': 'Área do lote (medido em Square Feet)',
    },
    {
        'name': 'Street',
        'category': 'symmetric_binary',
        'description': 'Tipo de rua de acesso a propriedade',
    },
    {
        'name': 'Alley',
        'category': 'nominal_categorical',
        'description': 'Tipo de beco de acesso à propriedade',
    },
    {
        'name': 'LotShape',
        'category': 'nominal_categorical',
        'description': 'Formato geral da propriedade',
    },
    {
        'name': 'LandContour',
        'category': 'nominal_categorical',
        'description': 'Planicidade da propriedade',
    },
    {
        'name': 'Utilities',
        'category': 'ordinal_categorical',
        'description': 'Tipos de utilidades disponiveis',
    },
    {
        'name': 'LotConfig',
        'category': 'nominal_categorical',
        'description': 'Configuração do lote',
    },
    {
        'name': 'LandSlope',
        'category': 'nominal_categorical',
        'description': 'Inclinação da propriedade',
    },
    {
        'name': 'Neighborhood',
        'category': 'nominal_categorical',
        'description': 'Localizações físicas dentro dos limites da cidade de Ames',
    },
    {
        'name': 'Condition1',
        'category': 'nominal_categorical',
        'description': 'Proximidade da residência à diferentes condições na cidade',
    },
    {
        'name': 'Condition2',
        'category': 'nominal_categorical',
        'description': 'Proximidade da residência à diferentes condições na cidade (caso mais de uma esteja presente)',
    },
    {
        'name': 'BldgType',
        'category': 'nominal_categorical',
        'description': 'Tipo de habitação',
    },
    {
        'name': 'HouseStyle',
        'category': 'nominal_categorial',
        'description': 'Estilo da habitação',
    },
    {
        'name': 'OverallQual',
        'category': 'ordinal_categorical',
        'description': 'Avalia o material total e o acabamento da casa',
    },
    {
        'name': 'OverallCond',
        'category': 'ordinal_categorical',
        'description': 'Avalia a condição geral da casa',
    },
    {
        'name': 'YearBuilt',
        'category': 'discrete_numerical',
        'description': 'Data em que a casa foi construída',
    },
    {
        'name': 'YearRemodAdd',
        'category': 'discrete_numerical',
        'description':'Data em que a casa foi remodelada (idêntico à data de construção se não houverem remodelações ou acréscimos à residência)',
    },
    {
        'name': 'RoofStyle',
        'category': 'nominal_categorical',
        'description': 'Tipo de telhado',
    },
    {
        'name': 'RoofMatl',
        'category': 'nominal_categorical',
        'description': 'Material do telhado',
    },
    {
        'name': 'Exterior1st',
        'category': 'nominal_categorical',
        'description': 'Revestimento exterior em casa',
    },
    {
        'name': 'Exterior2nd',
        'category': 'nominal_categorical',
        'description': 'Revestimento exterior em casa (se há mais de um material)',
    },
    {
        'name': 'MasVnrType',
        'category': 'nominal_categorical',
        'description': 'Tipo de folheado da alvenaria',
    },
    {
        'name': 'MasVnrArea',
        'category': 'continuous_numerical',
        'description': 'Área de folheado da alvenaria (medido em Square Feet)',
    },
    {
        'name': 'ExterQual',
        'category': 'ordinal_categorical',
        'description': 'Avalia a qualidade do material no exterior da casa'
    },
    {
        'name': 'ExterCond',
        'category': 'ordinal_categorical',
        'description': 'Avalia a condição atual do material no exterior da casa',
    },
    {
        'name': 'Foundation',
        'category': 'nominal_categorical',
        'description': 'Tipo de fundação da casa',
    },
    {
        'name': 'BsmtQual',
        'category': 'ordinal_categorical',
        'description': 'Avalia a altura do porão',
    },
    {
        'name': 'BsmtCond',
        'category': 'ordinal_categorical',
        'description': 'Avalia a condição geral do porão',
    },
    {
        'name': 'BsmtExposure',
        'category': 'ordinal_categorical',
        'description': 'Refere-se a walkout ou paredes no nível do jardim',
    },
    {
        'name': 'BsmtFinType1',
        'category': 'ordinal_categorical',
        'description': 'Classificação da área finalizada do porão'
    },
    {
        'name': 'BsmtFinSF1',
        'category': 'discrete_numerical',
        'description': 'Área finalizada Tipo 1 (medida em Square Feet)',
    },
    {
        'name': 'BsmtFinType2',
        'category': 'ordinal_categorical',
        'description': 'Classificação da área finalizada do porão (caso tenham múltiplos tipos)',
    },
    {
        'name': 'BsmtFinSF2',
        'category': 'discrete_numerical',
        'description': 'Área finalizada Tipo 1 (medida em Square Feet)',
    },
    {
        'name': 'BsmtUnfSF',
        'category': 'discrete_numerical',
        'description': 'Área não finalizada do porão (medida em Square Feet)',
    },
    {
        'name': 'TotalBsmtSF',
        'category': 'discrete_numerical',
        'description': 'Área total do porão (medida em Square Feet)',
    },
    {
        'name': 'Heating',
        'category': 'nominal_categorical',
        'description': 'Tipo de Aquecimento',
    },
    {
        'name': 'HeatingQC',
        'category': 'ordinal_categorical',
        'description': 'Qualidade e condição do aquecimento',
    },
    {
        'name': 'CentralAir',
        'category': 'asymmetric_binary',
        'description': 'Ar condicionado central',
    },
    {
        'name': 'Electrical',
        'category': 'nominal_categorical',
        'description': 'Sistema elétrico',
    },
    {
        'name': '1stFlrSF',
        'category': 'discrete_numerical',
        'description': 'Área do primeiro andar (medido em Square Feet)',
    },
    {
        'name': '2ndFlrSF',
        'category': 'discrete_numerical',
        'description': 'Área do segundo andar (medida em Square Feet)',
    },
    {
        'name': 'LowQualFinSF',
        'category': 'discrete_numerical',
        'description': 'Área finalizada em baixa qualidade (medida em Square Feet)',
    },
    {
        'name': 'GrLivArea',
        'category': 'discrete_numerical',
        'description': 'Área habitável acima do nível do solo (medida em Square Feet)',
    },
    {
        'name': 'BsmtFullBath',
        'category': 'discrete_numerical',
        'description': 'Número de banheiros no porão',
    },
    {
        'name': 'BsmtHalfBath',
        'category': 'discrete_numerical',
        'description': 'Número de banheiros de hóspede no porão',
    },
    {
        'name': 'FullBath',
        'category': 'discrete_numerical',
        'description': 'Número de banheiros',
    },
    {
        'name': 'HalfBath',
        'category': 'discrete_numerical',
        'description': 'Número de banheiros de hóspede',
    },
    {
        'name': 'BedroomAbvGr',
        'category': 'discrete_numerical',
        'description': 'Número de quartos acima do nível do solo (Não inclui quartos no porão)',
    },
    {
        'name': 'KitchenAbvGr',
        'category': 'discrete_numerical',
        'description': 'Número de cozinhas acima do nível do solo',
    },
    {
        'name': 'KitchenQual',
        'category': 'ordinal_categorical',
        'description': 'Qualidade da cozinha',
    },
    {
        'name': 'TotRmsAbvGrd',
        'category': 'discrete_numerical',
        'description': 'Número total de cômodos acima do nível do solo (não inclui banheiros)',
    },
    {
        'name': 'Functional',
        'category': 'nominal_categorical',
        'description': 'Funcionalidade da residência (É assumida funcionalidade típica até serem requisitadas investigações)',
    },
    {
        'name': 'Fireplaces',
        'category': 'discrete_numerical',
        'description': 'Número de lareiras na residência',
    },
    {
        'name': 'FireplaceQu',
        'category': 'ordinal_categorical',
        'description': 'Qualidade das lareiras',
    },
    {
        'name': 'GarageType',
        'category': 'nominal_categorical',
        'description': 'Localização da garagem',
    },
    {
        'name': 'GarageYrBlt',
        'category': 'discrete_numerical',
        'description': 'Ano de construção da garagem',
    },
    {
        'name': 'GarageFinish???? (ordinal_categorical)',
        'category': 'nominal_categorical',
        'description': 'Acabamento interno da garagem',
    },
    {
        'name': 'GarageCars',
        'category': 'discrete_numerical',
        'description': 'Tamanho da garagem (medida em capacidade para armazenar carros)',
    },
    {
        'name': 'GarageArea',
        'category': 'discrete_numerical',
        'description': 'Tamanho da garagem (medida em Square Feet)',
    },
    {
        'name': 'GarageQual',
        'category': 'ordinal_categorical',
        'description': 'Qualidade da garagem',
    },
    {
        'name': 'GarageCond',
        'category': 'ordinal_categorical',
        'description': 'Condição da garagem',
    },
    {
        'name': 'PavedDrive',
        'category': 'nominal_categorical',
        'description': 'Pavimento da entrada da garagem',
    },
    {
        'name': 'WoodDeckSF',
        'category': 'discrete_numerical',
        'description': 'Área do deck de madeira (medida em Square Feet)',
    },
    {
        'name': 'OpenPorchSF',
        'category': 'discrete_numerical',
        'description': 'Área da varanda aberta (medida em Square Feet)',
    },
    {
        'name': 'EnclosedPorch',
        'category': 'discrete_numerical',
        'description': 'Área da varanda fechada (medida em Square Feet)',
    },
    {
        'name': '3SsnPorch',
        'category': 'discrete_numerical',
        'description': 'Área da varanda de três estações(medida em Square Feet)',
    },
    {
        'name': 'ScreenPorch',
        'category': 'discrete_numerical',
        'description': 'Área da varanda com telas (medida em Square Feet)',
    },
    {
        'name': 'PoolArea',
        'category': 'discrete_numerical',
        'description': 'Área da piscina (medida em Square Feet)',
    },
    {
        'name': 'PoolQC',
        'category': 'ordinal_categorical',
        'description': 'Qualidade da piscina',
    },
    {
        'name': 'Fence',
        'category': 'ordinal_categorical',
        'description': 'Qualidade da cerca',
    },
    {
        'name': 'MiscFeature',
        'category': 'nominal_categorical',
        'description': 'Recurso variado que não se encaixa em outras categorias',
    },
    {
        'name': 'MiscVal',
        'category': 'continuous_numerical',
        'description': 'Valor monetário do recurso variado (medido em $)',
    },
    {
        'name': 'MoSold',
        'category': 'discrete_numerical',
        'description': 'Mês de venda (MM)',
    },
    {
        'name': 'YrSold',
        'category': 'discrete_numerical',
        'description': 'Ano de venda (YYYY)',
    },
    {
        'name': 'SaleType',
        'category': 'nominal_categorical',
        'description': 'Tipo da venda',
    },
    {
        'name': 'SaleCondition',
        'category': 'nominal_categorical',
        'description': 'Condição para venda',
    },
    {
        'name': 'SalePrice',
        'category': 'continuous_numerical',
        'description': 'Valor da venda (medido em $)',
    },
]

In [ ]:
# Imprimindo as informações de cada atributo
for attribute in attributes:
    print(f"{attribute['name']}")
    print("Categoria: ", end = "")

    if attribute["category"] == "nominal_categorical":
        print("Categórico Nominal")
    elif attribute["category"] == "ordinal_categorical":
        print("Categórico Ordinal")
    elif attribute["category"] == "discrete_numerical":
        print("Numérico Discreto")
    elif attribute["category"] == "continuous_numerical":
        print("Numérico Contínuo")
    elif attribute["category"] == "symmetric_binary":
        print("Binário Simétrico")
    elif attribute["category"] == "asymmetric_binary":
        print("Binário Assimétrico")

    print(f"Descrição: {attribute['description']}")
    print(f"Exemplos de valores: {df[attribute['name']].unique()[:5]}\n")

In [ ]:
# Compondo listas contendo os nomes dos atributos de cada categoria
nominal_categorical = []
ordinal_categorical = []
discrete_numerical = []
continuous_numerical = []
symmetric_binary = []
asymmetric_binary = []

for attribute in attributes:
    if attribute["category"] == "nominal_categorical":
        nominal_categorical.append(attribute['name'])
    elif attribute["category"] == "ordinal_categorical":
        ordinal_categorical.append(attribute['name'])
    elif attribute["category"] == "discrete_numerical":
        discrete_numerical.append(attribute['name'])
    elif attribute["category"] == "continuous_numerical":
        continuous_numerical.append(attribute['name'])
    elif attribute["category"] == "symmetric_binary":
        symmetric_binary.append(attribute['name'])
    elif attribute["category"] == "asymmetric_binary":
        asymmetric_binary.append(attribute['name'])

Ao traçar os possíveis valores dos atributos numéricos em um gráfico *boxplot*, podemos observar que, por exemplo, valores superiores a 200 libras para o custo de bilhetes são *outliers*.

In [ ]:
def probability(qtd, total):
  return qtd/total

# Questão 1.e

In [ ]:
correlation = df.corr(numeric_only=True)
plot = sn.heatmap(correlation, annot = True, fmt=".1f", linewidths=.6)
plot

Primeiramente, os valores dos coeficientes de correlação de Pearson apresentados no mapa de calor dizem o quão correlacionados dois atributos são: quanto mais próximos de 1 ou -1 o coeficiente for, mais correlacionados são os atributos.

É possível observar no mapa de calor que os atributos que possuem a maior correlação são o valor pago pelo bilhete do Titanic e a classe do alojamento. Dado que, os assentos presentes na primeira classse, usualmente, são mais caros quando comparados com os assentos da terceira classe.

Também vemos uma correlação entre a quantidade de irmãos e esposas, com a quantidade de pais e filhos que um individuo tem abordo no navio. Isso pode acontecer pois muito indivíduos poderiam estar fazendo uma viagem em família.

E por último, temos uma correlação entre a idade do individuo e a classe de seu alojamento. Para essa, não conseguimos identificar uma explicação para a correlação.

#Questão 2.a

In [ ]:
df['Cabin'] = df['Cabin'].fillna('')
df['Age'] = df['Age'].fillna(df['Age'].mean())
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode())

O conjunto de dados possui três colunas que contêm valores nulos: **cabin**, **age**, e **embarked**. Então somente foi tratado esses valores.

Para o atributo **cabin** foi substituído os valores nulos por uma string vazia, já que o atributo é uma string que não possui um padrão.

Para o atributo **age** o valor substituído sob os nulos foi a média da idade.

E para o atributo **embarked**, os valores nulos foram substituídos pela moda dos valores no conjunto de dados.

# Questão 2.b

In [ ]:
pd.cut(df.Fare, bins=3, right=True)

O único atributo que possui ruído é o **fare** que representa o preço dos bilhetes. Como o navio possui três classes, então dividimos os preços em três categorias eliminando assim qualquer ruído.

# Questão 2.c

A inconsistência presente no atributo Ticket não pode ser solucionada dado que seria extremamente custoso a padronização de todos os valores, além disso esse atributo não afeta a análise à ser realizada posteriormente.

In [ ]:
df[df['Fare'] == 0]

In [ ]:
def medium_fare(pclass):
  return df[df['Pclass'] == pclass]['Fare'].mean()

print(f'Preço médio do bilhete da classe 1: {medium_fare(1)}')
print(f'Preço médio do bilhete da classe 2: {medium_fare(2)}')
print(f'Preço médio do bilhete da classe 3: {medium_fare(3)}')

In [ ]:
pclasses = [1, 2, 3]

for pclass in pclasses:
  mean = medium_fare(pclass)
  df.loc[(df['Fare'] == 0) & (df['Pclass'] == pclass)].replace(0, mean)

A inconsistência presente no atributo Fare foi solucionada a partir da média dos valores pagos por individuos com o mesmo tipo de classe da passagem, para então substituir aqueles que se encontram zerados.

# Questão 2.d

In [ ]:
nominal_attributes = []

for index, row in df.iterrows():
  nominal_attributes.append([row["Name"], row["Ticket"], row["Cabin"]])

nominal_attributes = OneHotEncoder().fit_transform(df).toarray()
print(f'Vetor numérico contendo a transformação dos atributos categóricos nominais: {nominal_attributes}')

# Questão 2.e

O atributo **PassengerId** não necessita de uma transformação para ser interpretado como um atributo numérico, já que ele identifica unicamente cada indivíduo, começando pelo valor 1 e sendo incrementado em 1 para cada registro.

Similiarmente, o atributo **Pclass** também não necessita de uma transformação, já que cada classe econômica é representada por um número do intervalo [1, 3], onde 1 é a classe mais alta e 3 é a mais baixa.

In [ ]:
print(f'Tipo da coluna PassengerId: {df["PassengerId"].dtypes}')
print(f'Tipo da coluna Pclass: {df["Pclass"].dtypes}')

# Questão 2.f

In [ ]:
for attribute in numeric_attributes:
  df[attribute] = (df[attribute] - df[attribute].mean()) / df[attribute].std()
  print(f'Exemplos de valores normalizados para a coluna {attribute}: {df[attribute].unique()[:5]}\n')

In [ ]:
import sklearn
import numpy as np
import pandas as pd
import seaborn as sn

from sklearn.preprocessing import OneHotEncoder

In [ ]:
# Carregando o dataset de treino
df = pd.read_csv('./train.csv')
print(df)

# Questão 1.a


*MSSubClass:* 

Identifies the type of dwelling involved in the sale.

**tipo:**

*MSZoning:* 

Identifies the general zoning classification of the sale.

**tipo:**

*LotFrontage:*

Linear feet of street connected to property

**tipo:**

*LotArea:*

Lot size in square feet

**tipo:**

*Street:*

Type of road access to property

**tipo:**

*Alley:* 

Type of alley access to property

**tipo:**

*LotShape:*

General shape of property

**tipo:**

*LandContour:*

Flatness of the property

**tipo:**

*Utilities:*

Type of utilities available

**tipo:**

*LotConfig:*

Lot configuration

**tipo:**

*LandSlope:*

Slope of property

**tipo:**

*Neighborhood:* 

Physical locations within Ames city limits			

**tipo:**

*Condition1:* 

Proximity to various conditions

**tipo:**

*Condition2:* 

Proximity to various conditions (if more than one is present)

**tipo:**

*BldgType:*

Type of dwelling

**tipo:**

*HouseStyle:*

Style of dwelling

**tipo:**

*OverallQual:*

Rates the overall material and finish of the house

**tipo:**

*OverallCond:*

Rates the overall condition of the house

**tipo:**

*YearBuilt:*

Original construction date

**tipo:**

*YearRemodAdd:*

Remodel date (same as construction date if no remodeling or additions)

**tipo:**

*RoofStyle: Type of roof

**tipo:**

*RoofMatl: Roof material

**tipo:**

*Exterior1st: Exterior covering on house

**tipo:**

*Exterior2nd: Exterior covering on house (if more than one material)

**tipo:**

*MasVnrType: Masonry veneer type

**tipo:**

*MasVnrArea: Masonry veneer area in square feet

**tipo:**

*ExterQual: Evaluates the quality of the material on the exterior 		

**tipo:**

*ExterCond: Evaluates the present condition of the material on the exterior

**tipo:**

*Foundation: Type of foundation

**tipo:**

*BsmtQual: Evaluates the height of the basement

**tipo:**

*BsmtCond: Evaluates the general condition of the basement

**tipo:**

*BsmtExposure: Refers to walkout or garden level walls

**tipo:**

*BsmtFinType1: Rating of basement finished area

**tipo:**

*BsmtFinSF1: Type 1 finished square feet

**tipo:**

*BsmtFinType2: Rating of basement finished area (if multiple types)

**tipo:**

*BsmtFinSF2: Type 2 finished square feet

**tipo:**

*BsmtUnfSF: Unfinished square feet of basement area

**tipo:**

*TotalBsmtSF: Total square feet of basement area

**tipo:**

*Heating: Type of heating

**tipo:**

*HeatingQC: Heating quality and condition

**tipo:**

*CentralAir: Central air conditioning

**tipo:**

*Electrical: Electrical system		

**tipo:**

*1stFlrSF: First Floor square feet 

**tipo:**

*2ndFlrSF: Second floor square feet

**tipo:**

*LowQualFinSF: Low quality finished square feet (all floors)

**tipo:**

*GrLivArea: Above grade (ground) living area square feet

**tipo:**

*BsmtFullBath: Basement full bathrooms

**tipo:**

*BsmtHalfBath: Basement half bathrooms

**tipo:**

*FullBath: Full bathrooms above grade

**tipo:**

*HalfBath: Half baths above grade

**tipo:**

*Bedroom: Bedrooms above grade (does NOT include basement bedrooms)

**tipo:**

*Kitchen: Kitchens above grade

**tipo:**

*KitchenQual: Kitchen quality

**tipo:**

*TotRmsAbvGrd: Total rooms above grade (does not include bathrooms)

**tipo:**

*Functional: Home functionality (Assume typical unless deductions are warranted)

**tipo:**
*Fireplaces: Number of fireplaces

**tipo:**
*FireplaceQu: Fireplace quality

**tipo:**
*GarageType: Garage location	

**tipo:**
*GarageYrBlt: Year garage was built		

**tipo:**

*GarageFinish: Interior finish of the garage

**tipo:**

*GarageCars: Size of garage in car capacity

**tipo:**

*GarageArea: Size of garage in square feet

**tipo:**

*GarageQual: Garage quality

**tipo:**

*GarageCond: Garage condition		

**tipo:**

*PavedDrive: Paved driveway

**tipo:**

*WoodDeckSF: Wood deck area in square feet

**tipo:**

*OpenPorchSF: Open porch area in square feet

**tipo:**

*EnclosedPorch: Enclosed porch area in square feet

**tipo:**

*3SsnPorch: Three season porch area in square feet

**tipo:**

*ScreenPorch: Screen porch area in square feet

**tipo:**

*PoolArea: Pool area in square feet

**tipo:**

*PoolQC: Pool quality

**tipo:**

*Fence: Fence quality

**tipo:**

*MiscFeature: Miscellaneous feature not covered in other categories

**tipo:**

*MiscVal: $Value of miscellaneous feature

**tipo:**

*MoSold: Month Sold (MM)

**tipo:**

*YrSold: Year Sold (YYYY)

**tipo:**

*SaleType: Type of sale		

**tipo:**

*SaleCondition: Condition of sale

**tipo:**
